In [111]:
from ipycanvas import Canvas, hold_canvas
import random
import time
import matplotlib.pyplot as plt

cell_pixel_size = 5
width = 100
height = 100
threshold = 10
num_threshold_bases = 25

canvas = Canvas(width=width*cell_pixel_size, height=height*cell_pixel_size)
display(canvas)


def populate(width, height):
    world = []
    for i in range(height):
        a = []
        for j in range(width):
            a.append(random.randint(0, 1))
        world.append(a)
    
    return world


def alive_next_generation(neighborhood):
    num_of_neighbors = sum(sum(column) for column in neighborhood) - neighborhood[1][1]


    # 1: En levende celle med færre enn to naboer dør av "ensomhet"
    if neighborhood[1][1] == 1 and num_of_neighbors < 2:
        return False

    # 2: En levende celle med to eller tre naboer overlever til neste generasjon.
    if neighborhood[1][1] == 1 and (num_of_neighbors == 2 or num_of_neighbors == 3):
        return True

    # 3: En levende celle med flere enn tre naboer dør av "ressursmangel"
    if neighborhood[1][1] == 1 and num_of_neighbors > 3:
        return False

    # 4: Det oppstår liv i en celle uten liv hvis den har nøyaktig tre naboer.
    if neighborhood[1][1] == 0 and num_of_neighbors == 3:
        return True

    # return False
    

def next_generation(world):
    new_world = [[cell for cell in row] for row in world]
    # print(new_world)
    # new_world = world.copy()
    for i in range(height):
        for j in range(width):
            statement = alive_next_generation(find_neighborhood(world, i, j))
            if statement:
                new_world[i][j] = 1
            else:
                new_world[i][j] = 0
    return new_world


def find_neighborhood(world, y, x):
    global height
    global width
    neighborhood = []
    for i in range(y - 1, y + 2):
        a = []
        for j in range(x - 1, x + 2):
            if i >= height and j >= width:
                a.append(world[0][0])
            elif i >= height:
                a.append(world[0][j])
            elif j >= width:
                a.append(world[i][0])
            else:
                a.append(world[i][j])
        neighborhood.append(a)
    return neighborhood

def population(world):
    global height
    global width
    s = 0
    for _ in world:
        s += sum(_)
    return s


world = populate(width, height)
delta_population = 0
prev_population = 0
delta_list = []
generations = []
population_generations = []
d = 0

for i in range(num_threshold_bases):
    delta_list.append(num_threshold_bases*10)

# while True:
# for x in range(1000):
while sum(delta_list)/num_threshold_bases >= threshold:
    d += 1
    with hold_canvas():
        # # Draw canvas
        canvas.clear()

        for i in range(height):
            for j in range(width):
                if world[i][j] == 1:
                    canvas.fill_style = "black"
                    canvas.fill_rect(cell_pixel_size*j, cell_pixel_size*i, cell_pixel_size)
                else:
                    continue
        # time.sleep(1/1000)
    
    world = next_generation(world)
    population_generations.append(population(world))
    delta_population = abs(population_generations[-1] - population_generations[len(population_generations) - 2])
    delta_list.pop(0)
    delta_list.append(delta_population)
    # print(f"{population_generations[-1]}\t{delta_population}")
    generations.append(d)
    
plt.plot(generations, population_generations)
plt.show()

    

Canvas(width=500)

2605	0
2396	209
2291	105
2263	28
2180	83
2124	56
2073	51
2111	38
2006	105
1970	36
1933	37
1846	87
1854	8
1748	106
1740	8
1638	102
1701	63
1677	24
1670	7
1554	116
1633	79
1478	155
1426	52
1366	60
1349	17
1273	76
1225	48
1209	16
1146	63
1230	84
1162	68
1158	4
1139	19
1115	24
1076	39
1065	11
1062	3
1111	49
1068	43
1077	9
1049	28
1104	55
1091	13
1074	17
988	86
1032	44
1014	18
1013	1
982	31
968	14
977	9
922	55
923	1
900	23
889	11
857	32
891	34
875	16
887	12
869	18
879	10
878	1
886	8
859	27
881	22
876	5
860	16
902	42
869	33
898	29
901	3
918	17
865	53
863	2
864	1
854	10
859	5
872	13
858	14
866	8
937	71
903	34
946	43
866	80
900	34
899	1
907	8
914	7
870	44
900	30
898	2
887	11
884	3
859	25
870	11
872	2
825	47
813	12
827	14
804	23
837	33
757	80
787	30
770	17
808	38
756	52
797	41
762	35
796	34
783	13
786	3
781	5
805	24
818	13
769	49
802	33
774	28
758	16
785	27
803	18
830	27
811	19
847	36
863	16
855	8
878	23
834	44
847	13
805	42
802	3
798	4
810	12
787	23
815	28
797	18
714	83
717	3
704	13
691	13
639